In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from torch.utils.data import Dataset, DataLoader
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip install pycocotools -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torch import nn, optim
import json

In [4]:
# Параметры
data_dir = '/kaggle/input/animalscropped'
train_dir = f'{data_dir}/train'
val_dir = f'{data_dir}/valid'
batch_size = 8
num_epochs = 60
num_classes = 1  # Установите количество классов в вашем датасете

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
class CocoDataset(Dataset):
    def __init__(self, img_dir, json_file, transform=None):
        with open(json_file) as f:
            self.coco_data = json.load(f)
        self.img_dir = img_dir
        self.transform = transform

        # Создаем индекс аннотаций по id изображения для быстрого доступа
        self.annotations_index = {}
        for annotation in self.coco_data['annotations']:
            image_id = annotation['image_id']
            if image_id not in self.annotations_index:
                self.annotations_index[image_id] = []
            self.annotations_index[image_id].append(annotation)

    def __len__(self):
        return len(self.coco_data['images'])

    def __getitem__(self, idx):
        img_info = self.coco_data['images'][idx]
        img_path = os.path.join(self.img_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')

        label = None
        # Получаем аннотации для текущего изображения
        if img_info['id'] in self.annotations_index:
            annotations = self.annotations_index[img_info['id']]
            for annotation in annotations:
                label = 0 if annotation["category_id"] == 1 else 1
        if self.transform:
            # image = self.transform(np.array(image))
            # mask = self.transform(np.array(mask_image))
            image = self.transform(image)
        return image, torch.tensor(label).float().unsqueeze(0)

In [7]:
# Загрузка данных COCO
train_dataset = CocoDataset(train_dir, f'{train_dir}/_annotations.coco.json', transform=transform)
val_dataset = CocoDataset(val_dir, f'{val_dir}/_annotations.coco.json', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
model = torchvision.models.resnet101(pretrained=True)

# Изменение выходного слоя для бинарной классификации
model.fc = nn.Linear(model.fc.in_features, 1)  # Один нейрон для бинарной классификации

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.BCEWithLogitsLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 204MB/s]  


In [9]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    epoch_loss = running_loss 
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    return running_loss/len(train_loader)

# Функция валидации
def validate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, targets in val_loader:
            targets = targets.to(device)

            images = images.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += len(targets)
            correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(train_loader) 
    accuracy = correct / total
    return epoch_loss, accuracy

In [10]:
# Цикл обучения
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy = validate(model, val_loader, criterion)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

print("Training complete.")

Epoch [1/60], Loss: 0.6935
Epoch 1/60, Train Loss: 0.6935, Val Loss: 0.1890, Val Accuracy: 0.6040
Epoch [2/60], Loss: 0.6017
Epoch 2/60, Train Loss: 0.6017, Val Loss: 0.1550, Val Accuracy: 0.7360
Epoch [3/60], Loss: 0.6018
Epoch 3/60, Train Loss: 0.6018, Val Loss: 0.1741, Val Accuracy: 0.7472
Epoch [4/60], Loss: 0.5997
Epoch 4/60, Train Loss: 0.5997, Val Loss: 0.1544, Val Accuracy: 0.7360
Epoch [9/60], Loss: 0.5138
Epoch 9/60, Train Loss: 0.5138, Val Loss: 0.1468, Val Accuracy: 0.7494
Epoch [10/60], Loss: 0.5333
Epoch 10/60, Train Loss: 0.5333, Val Loss: 0.1400, Val Accuracy: 0.7494
Epoch [11/60], Loss: 0.4984
Epoch 11/60, Train Loss: 0.4984, Val Loss: 0.1664, Val Accuracy: 0.7181
Epoch [12/60], Loss: 0.4843
Epoch 12/60, Train Loss: 0.4843, Val Loss: 0.1317, Val Accuracy: 0.7629
Epoch [13/60], Loss: 0.4526
Epoch 13/60, Train Loss: 0.4526, Val Loss: 0.2276, Val Accuracy: 0.6779
Epoch [14/60], Loss: 0.4317
Epoch 14/60, Train Loss: 0.4317, Val Loss: 0.1521, Val Accuracy: 0.7517
Epoch [15/

In [11]:
torch.save(model, "/kaggle/working/saved.pth")